<a href="https://colab.research.google.com/github/Randomz4-debug/Chatapp/blob/main/ML_using_pandas_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Machine Learning***
(pandas and sklearn)
---

In [5]:
import requests

url = "https://gist.githubusercontent.com/yanyanzheng96/8fe13cc4b7c6a1391d4841018340a627/raw/melb_data.csv"  # web link to CSV
response = requests.get(url)

text = response.content.decode("utf-8")
text = text.replace("Bedroom2", "Bedroom")

with open("house_prices.csv", "w", encoding="utf-8") as file:
    file.write(text)

print("CSV downloaded successfully")


CSV downloaded successfully


In [6]:
import pandas as pd

file_csv = "house_prices.csv"
data = pd.read_csv(file_csv)
#data.describe()
data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [7]:
prediction = data.Price
features = ['Rooms', 'Bedroom', 'Bathroom', 'Longtitude', 'Lattitude', 'Landsize', 'BuildingArea', 'YearBuilt']
# The Melbourne data has some missing values (some houses for which some variables weren't recorded.)
# We'll learn to handle missing values in a later tutorial.
# Your Iowa data doesn't have missing values in the columns you use.
# So we will take the simplest option for now, and drop houses from our data.
# Don't worry about this much for now, though the code is:

# dropna drops missing values (think of na as "not available")
data = data.dropna(axis=0)

x = data[features]
y = data.Price
x.describe()

,Rooms,Bedroom,Bathroom,Longtitude,Lattitude,Landsize,BuildingArea,YearBuilt
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,2.902034,1.576340,144.990201,-37.807904,471.006940,141.568645,1964.081988
std,0.971079,0.970055,0.711362,0.099165,0.075850,897.449881,90.834824,38.105673
min,1.000000,0.000000,1.000000,144.542370,-38.164920,0.000000,0.000000,1196.000000
25%,2.000000,2.000000,1.000000,144.926198,-37.855438,152.000000,91.000000,1940.000000
50%,3.000000,3.000000,1.000000,144.995800,-37.802250,373.000000,124.000000,1970.000000
75%,4.000000,3.000000,2.000000,145.052700,-37.758200,628.000000,170.000000,2000.000000
max,8.000000,9.000000,8.000000,145.526350,-37.457090,37000.000000,3112.000000,2018.000000


In [8]:
x.head()

,Rooms,Bedroom,Bathroom,Longtitude,Lattitude,Landsize,BuildingArea,YearBuilt
1,2,2.0,1.0,144.9934,-37.8079,156.0,79.0,1900.0
2,3,3.0,2.0,144.9944,-37.8093,134.0,150.0,1900.0
4,4,3.0,1.0,144.9941,-37.8072,120.0,142.0,2014.0
6,3,4.0,2.0,144.9993,-37.8024,245.0,210.0,1910.0
7,2,2.0,1.0,144.9954,-37.8060,256.0,107.0,1890.0


In [9]:
from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
model = DecisionTreeRegressor(random_state=1)

# Fit model
model.fit(x, y)

DecisionTreeRegressor(random_state=1)

In [10]:
#Maing decisions for 5 sets:
print("Making predictions for the following 5 houses:")
print(x.head())
print("The predictions are")
print(model.predict(x.head()))

Making predictions for the following 5 houses:
   Rooms  Bedroom  Bathroom  Longtitude  Lattitude  Landsize  BuildingArea  \
1      2      2.0       1.0    144.9934   -37.8079     156.0          79.0   
2      3      3.0       2.0    144.9944   -37.8093     134.0         150.0   
4      4      3.0       1.0    144.9941   -37.8072     120.0         142.0   
6      3      4.0       2.0    144.9993   -37.8024     245.0         210.0   
7      2      2.0       1.0    144.9954   -37.8060     256.0         107.0   

   YearBuilt  
1     1900.0  
2     1900.0  
4     2014.0  
6     1910.0  
7     1890.0  
The predictions are
[1035000. 1465000. 1600000. 1876000. 1636000.]


In [11]:
from sklearn.metrics import mean_absolute_error

predicted_prices = model.predict(x)
err = mean_absolute_error(y, predicted_prices)

In [12]:
'''The Problem with "In-Sample" Scores
The measure we just computed can be called an "in-sample" score. We used a single "sample" of houses for both building the model and evaluating it. Here's why this is bad.

Imagine that, in the large real estate market, door color is unrelated to home price.

However, in the sample of data you used to build the model, all homes with green doors were very expensive. The model's job is to find patterns that predict home prices, so it will see this pattern, and it will always predict high prices for homes with green doors.

Since this pattern was derived from the training data, the model will appear accurate in the training data.

But if this pattern doesn't hold when the model sees new data, the model would be very inaccurate when used in practice.

Since models' practical value come from making predictions on new data, we measure performance on data that wasn't used to build the model. The most straightforward way to do this is to exclude some data from the model-building process, and then use those to test the model's accuracy on data it hasn't seen before. This data is called validation data.

Coding It
The scikit-learn library has a function train_test_split to break up the data into two pieces. We'll use some of that data as training data to fit the model, and we'll use the other data as validation data to calculate mean_absolute_error.

Here is the code:'''

from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
train_X, val_X, train_y, val_y = train_test_split(x, y, random_state = 0)
# Define model
model = DecisionTreeRegressor()
# Fit model
model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

259566.3118140736
